<a href="https://colab.research.google.com/github/ttheimeroo/ttheimeroo.github.io/blob/main/snstk_calib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Python Functions for SensTek Calibration
Tom Theimer 8/25/2023

This is a development and testing sheet for a Python module of functions to support calibration processing and moving of data to the SensTek database.

There is an associated test harness file "snstk_calib_py_TH.ipynb" (in the calibration folder) for verification of the individual module functions.

##Get a MySQL Connection - prod

In [ ]:

def get_mysql_connection():
  """The shell command
  !pip3 install mysql-connector-python
  must be run in the main program before calling this function
  """
  # MySQL setup
  import mysql.connector

  # Connection settings
  mysql_config = {
     "host": "data.snstk.com",
     "user": "colab1",
     "password": "3liLLLy&poot",
     "database": "sensordata"
  }

  # Create a MySQL connection
  return mysql.connector.connect(**mysql_config) # the double asterisk
  #                 unpacks the dictionary into key-value pairs which
  #                 it applies are arguments to a function.

##Get a MySQL Connection - CLONE

In [ ]:

def get_mysql_CLONE_connection():
  """The shell command
  !pip3 install mysql-connector-python
  must be run in the main program before calling this function
  """
  # MySQL setup
  import mysql.connector

  # Connection settings
  mysql_config = {
     "host": "data.snstk.com",
     "user": "colab1",
     "password": "3liLLLy&poot",
     "database": "sensorclone"
  }

  # Create a MySQL connection
  return mysql.connector.connect(**mysql_config) # the double asterisk
  #                 unpacks the dictionary into key-value pairs which
  #                 it applies are arguments to a function.

##Quotes in String or NULL
This preps a string for a MySQL INSERT statement. Included quotes or NULL for None

In [ ]:
# Function to prep a value for MySQL INSERT stmt. Return a value that includes quotes if needed.
def mysqlprep(insertvalstr):
  if insertvalstr == None:
    return "Null"
  elif (isinstance(insertvalstr, (int, float, complex)) or
        insertvalstr.isnumeric()):
    return insertvalstr
  else:
    return f"'{insertvalstr}'"

##Escape single quotes

In [ ]:
# Function to escape single quotes
def escape_single_quotes(input_string):
    return input_string.replace("'", "\\'")

##read_data_from_param_sprdsht()

In [ ]:
# This mount routine is required to alow connection to files in Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
"""
Function to read the Parameter file (xlsx) and move the test parameters into variables.
The input parameter is the full file path.
A dictionary is returned. Most keys refer to single data elements with the
exception of the keys "rs_pairs_df" and "buffer_start_stop_df" which refer to
dataframes.
"""
def read_data_from_param_sprdsht(ss_file_path):
  import pandas as pd
  df = pd.read_excel(ss_file_path, sheet_name='Seq_params', usecols="A:H")

  testdefn = {}
  testdefn['param_full_file_path'] = ss_file_path
  testdefn['testing_lab'] = df.iloc[df.iloc[:,0:1].isin(['testing_lab']).idxmax()[0] ,1]
  testdefn['timezone_utc_offset'] = df.iloc[df.iloc[:,0:1].isin(['timezone offset']).idxmax()[0] ,1]
  testdefn['technician'] = df.iloc[df.iloc[:,0:1].isin(['technician']).idxmax()[0] ,1]
  testdefn['we_source'] = df.iloc[df.iloc[:,0:1].isin(['WE source']).idxmax()[0] ,1]
  testdefn['re_source'] = df.iloc[df.iloc[:,0:1].isin(['RE source']).idxmax()[0] ,1]
  testdefn['test_type'] = df.iloc[df.iloc[:,0:1].isin(['test type']).idxmax()[0] ,1]
  testdefn['time_in_buffer_min'] = df.iloc[df.iloc[:,0:1].isin(['time in buffer']).idxmax()[0] ,1]
  testdefn['conditioning_time_min'] = df.iloc[df.iloc[:,0:1].isin(['conditioning time']).idxmax()[0] ,1]
  testdefn['conditioning_substance'] = df.iloc[df.iloc[:,0:1].isin(['cond. substance']).idxmax()[0] ,1]
  testdefn['test_date'] = df.iloc[df.iloc[:,0:1].isin(['Test Date']).idxmax()[0] ,1].date()
  testdefn['sample_sec_for_calcs'] = df.iloc[df.iloc[:,0:1].isin(['sample calc dur']).idxmax()[0] ,1]
  testdefn['sample_start_sec'] = df.iloc[df.iloc[:,0:1].isin(['sample calc start']).idxmax()[0] ,1]
  testdefn['comments'] = df.iloc[df.iloc[:,0:1].isin(['comments']).idxmax()[0] ,1]
  if not (isinstance(testdefn['time_in_buffer_min'], (int, float, complex)) or
          testdefn['time_in_buffer_min'].isnumeric()):
    testdefn['comments'] += f"\nThe duration in buffer was: \"{testdefn['time_in_buffer_min']}\""
    testdefn['time_in_buffer_min'] = None
  lists_row = df.iloc[:,0:1].isin(['Sensor & Buffer Tables']).idxmax()[0] + 2
  rs_pairs_df = df.iloc[lists_row:, 2:4].dropna(axis = 0, how = 'all')
  rs_pairs_df.columns = ['reader_sn','sensor_id']
  rs_pairs_df['reader_sn'] = rs_pairs_df['reader_sn'].astype(int)
  rs_pairs_df['sensor_id'] = rs_pairs_df['sensor_id'].astype(int)
  #testdefn['rs_pairs_dct'] = rs_pairs_df.to_dict(orient='records') #not used??
  testdefn['rs_pairs_df'] = rs_pairs_df

  buffer_start_stop_df = df.iloc[lists_row:,5:8].dropna(axis = 0, how = 'all')
  buffer_start_stop_df.columns = ['buffer_pH', 'start_time', 'stop_time']
  buffer_start_stop_df.sort_values(by=['start_time'], inplace=True)
  testdefn['buffer_start_stop_df'] = buffer_start_stop_df
  #testdefn['start_stop_dct'] = buffer_start_stop_df.to_dict(orient='records') #not used??
  testdefn['first_start_time'] = buffer_start_stop_df.iloc[0].at['start_time']
  testdefn['last_stop_time'] = buffer_start_stop_df.iloc[-1].at['stop_time']
  reader_ser = rs_pairs_df['reader_sn'] #This is for the next line
  testdefn['readerlststr'] = ','.join(map(str, reader_ser.tolist())) # used for documentatn

  return testdefn
read_data_from_param_sprdsht('/content/drive/Shared drives/SNSTK (Public)/5000 ENGINEERING/Reader/_TESTING/Calibration/202311/28/20231128_Calib_1x1341_1342_cycle_1_CUST.xlsx')

TypeError: ignored

##Push param data to DB

In [ ]:
"""
Push parameter data (from the spreadsheet) to the database table TstCycle.
(TCKey will be auto-populated)
@param "cnx" MySQL connection object.
@param "paramdct" Dictionary of test parameters.
"""
def params_to_sql(cnx, paramdct):
  # Create a cursor object to execute SQL queries
  cursor = cnx.cursor()

  param_full_file_path = paramdct['param_full_file_path']
  onetestdate = paramdct['test_date']
  onetestinglab = paramdct['testing_lab']
  onetech = paramdct['technician']
  onetesttype = paramdct['test_type']
  onesamplewait = paramdct['sample_start_sec']
  onesampledur = paramdct['sample_sec_for_calcs']
  onetestnotes = paramdct['comments']
  immersion_min_q = mysqlprep(paramdct['time_in_buffer_min'])
  cond_min = paramdct['conditioning_time_min']
  cond_sub = paramdct['conditioning_substance']
  first_start = paramdct['first_start_time']
  last_end = paramdct['last_stop_time']
  query = ("INSERT IGNORE into TstCycle " +
          "(TCSprdShtName,TCDate,TCLab,TCTech,TCType,TCDurInBuffer," +
          "TCConditioningMin,TCCondSubstance,TCSampleStart,TCSampleDur," +
          "TCFirstStartTime,TCLastEndTIme,TCNotes) VALUES " +
          f"('{param_full_file_path}','{onetestdate}','{onetestinglab}','{onetech}'," +
            f"'{onetesttype}',{immersion_min_q},{cond_min},'{cond_sub}'," +
            f"{onesamplewait},{onesampledur},'{first_start}','{last_end}'," +
            f"'{escape_single_quotes(onetestnotes)}');")
  print(f'From params_to_sql(): {query}')
  cursor.execute(query)
  #print("Inserting " + param_full_file_path)
  cnx.commit()
  cursor.execute('SELECT LAST_INSERT_ID();')
  last_tc_key = cursor.fetchone()[0]
  cursor.close()         # It should not be necessary to close and open the
  cursor = cnx.cursor()  # cursor but the following INSERTs are not working. Still not working... but no error.
  # Converting to Dict below because the loop to push the data to MySQL
  # was written for Dict structure. It can be refactored to loop through
  # the DataFrame.  Same with rs_pairs_dct below.
  start_stop_dct = paramdct['buffer_start_stop_df'].to_dict(orient='records')
  # start_stop_dct: [{'buffer_pH': 3, 'start_time': datetime.time(12, 17, 5), 'stop_time': datetime.time(12, 22, 5)}
  for onebuf in start_stop_dct:
    buf = onebuf['buffer_pH']
    starttime = onebuf['start_time'].strftime('%H:%M:%S')
    stoptime = onebuf['stop_time'].strftime('%H:%M:%S')
    query = ("INSERT IGNORE INTO `TstTimes` (TTTCFK,TTbufpH,TTstarttime,TTendtime) " +
            "VALUES" +
            f"({last_tc_key},{buf},'{starttime}','{stoptime}');")
    print(query)
    cursor.execute(query)
    cnx.commit()

  rs_pairs_dct = paramdct['rs_pairs_df'].to_dict(orient='records')
  # rs_pairs_dct: [{'reader_sn': 216, 'sensor_id': 1001204}, {'reader_sn': 236, 'sensor_id': 1001185}, {'reader_sn': 247, 'sensor_id': 1001161}]
  for onepair in rs_pairs_dct:
    readersn = onepair['reader_sn']
    sensorid = onepair['sensor_id']
    query = ("INSERT IGNORE INTO `RSPairs` (RSTCFK,RSRdrSNFK,RSSensFK) " +
            "VALUES" +
            f"({last_tc_key},{readersn},{sensorid});")
    print(query)
    cursor.execute(query)
    cnx.commit()

  # Close the cursor and connection
  cursor.close()
  print('Done.')

##Delimited str of Hub names

In [ ]:
"""
function to create a string of Hub names separated by vertical bars.
e.g. CUST|DEMO|BORON
This uses an established connection to the SSensTek MySQL database
which is provided to the function as the only parameter of the function.
It returns a string containing all of the Hub names separated by vertical bars.
The Hub names come from the from database. This string is used in a regex expression.
"""
def hubs_vert_bar_delim(cnx):
    try:
        # Connect to the database
        cursor = cnx.cursor()

        # Query the Hubs table for HubName column
        query = "SELECT HubName FROM Hubs"
        cursor.execute(query)

        # Fetch all rows
        rows = cursor.fetchall()

        # Combine the rows using vertical bar delimiter
        combined_string = '|'.join(row[0] for row in rows)

        return combined_string

    except mysql.connector.Error as err:
        print("Error: ", err)

    finally:
        # Close the cursor
        if cnx.is_connected():
            cursor.close()